<a href="https://colab.research.google.com/github/imabari/musen/blob/main/rakuten5G.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import datetime
import pathlib
import time
import urllib.parse

import pandas as pd
import requests

In [2]:
api = {
    # 1:免許情報検索  2: 登録情報検索
    "ST": 1,
    # 詳細情報付加 0:なし 1:あり
    "DA": 1,
    # スタートカウント
    "SC": 1,
    # 取得件数
    "DC": 2,
    # 出力形式 1:CSV 2:JSON 3:XML
    "OF": 1,
    # 無線局の種別
    "OW": "FB",
    # 所轄総合通信局
    # "IT": "A",
    # 免許人名称/登録人名称
    "NA": "楽天モバイル",
}

In [3]:
sc = 1
n = 0

dfs = []

while True:

    api["SC"] = sc
    parm = urllib.parse.urlencode(api, encoding="shift-jis")

    r = requests.get("https://www.tele.soumu.go.jp/musen/list", parm)
    r.raise_for_status()

    cr = csv.reader(r.text.splitlines(), delimiter=",")
    data = list(cr)

    n = int(data[0][1])

    df_tmp = pd.DataFrame(data[1:]).dropna(how="all")
    dfs.append(df_tmp)

    sc += 500

    print(sc, n)

    if sc > n:
        break
 
    time.sleep(10)

501 3666
1001 3666
1501 3666
2001 3666
2501 3666
3001 3666
3501 3666
4001 3666


In [4]:
df = pd.concat(dfs).reset_index(drop=True)

In [5]:
df[0] = df.index

In [6]:
df[23] = df[23].str.split(r"\\n")

In [7]:
df1 = df.explode(23).reset_index(drop=True)

In [8]:
df2 = df1[23].str.replace(" +", " ").str.split(r"\\t", expand=True).rename(columns={0: "電波の型式", 1: "周波数", 2: "空中線電力"})

In [9]:
df3 = df1.join(df2)

In [10]:
df3.to_csv("rakuten5G.csv", encoding="utf_8_sig")

In [11]:
df3["電波の型式"].value_counts()

100MX7W    6397
99M9X7W    1340
20M0X7W     726
Name: 電波の型式, dtype: int64

In [12]:
df3["周波数"].value_counts()

27.15 GHz       1600
27.24996 GHz    1599
27.34992 GHz    1599
27.05004 GHz    1599
3849.99 MHz     1340
1835 MHz         726
Name: 周波数, dtype: int64

In [13]:
df3["空中線電力"].value_counts()

663 mW       3400
442 mW       1885
221 mW       1068
40 W          726
218 mW        457
             ... 
22.313 W        1
91.795 W        1
126.691 W       1
16.023 W        1
6.561 W         1
Name: 空中線電力, Length: 76, dtype: int64